In [1]:
import pandas as pd
from sqlalchemy.types import VARCHAR, INTEGER, FLOAT
from sqlalchemy import text
from sqlalchemy import create_engine


In [2]:
# connect to the database
engine = create_engine('mssql+pyodbc://WagnerProdAGL1/Targit_DM?driver=ODBC+Driver+17+for+SQL+Server', echo=False, fast_executemany=True)

In [3]:
# Read OTIL excel
df_OTIF_Stage = pd.read_excel('Files\CCPA Order Detail Wagner December 2023.xlsx', sheet_name='CCPA Order Detail')

In [4]:
df_OTIF_Stage.head(5)

,Rpt Dt,Frcst Enty Dlr Cd,Main Dlr Cd,Dlr Cd Emer,Inv Str No,Str No,Sls Chan,End Cust No,End Cust Nm,End Cust Ord No,...,Cof Svc Elig Ind,Cof Non Elig Rsn,Otif Svc Elig Ind,Otif Non Elig Rsn,Otif Ind,Teps Ind,Ces Ind,Key Cat Account Num,Parent Customer No,Cost Center
0,2023-12-14,E250,E250,E260,09,09,WO,02901B,L G EVERIST INC,09S410327,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,CA
1,2023-12-01,E250,E250,E250,00,00,OTC,58362A,MARTIN MARIETTA MATERIALS,AGC571901,...,Y,NaN,Y,NaN,Y,NaN,NaN,CCHD,58362.0,DN
2,2023-12-18,E250,E250,E252,02,02,OTC,53818,LACY CONSTRUCTION COMPANY,02C567115,...,Y,NaN,Y,NaN,Y,N,NaN,NaN,NaN,2P
3,2023-12-11,E250,E250,E253,03,03,WO,88095,TOTAL TERRAIN INC,03S380488,...,Y,NaN,Y,NaN,N,N,NaN,NaN,NaN,MS
4,2023-12-15,E250,E250,E250,00,00,PS,31224,EQUIPMENT TRUCKS INC,AGC580589,...,Y,NaN,Y,NaN,Y,N,NaN,NaN,NaN,DN


### Logic to getting the calculations made

In [ ]:
# Calculate count of 'Y' in column 'OTIL LINES' for each dealer code
df_count_ones = df_OTIF_Stage[df_OTIF_Stage['Otif Ind'] == 'Y'].groupby('Dlr Cd Emer')['Otif Ind'].count().reset_index()
df_count_ones = df_count_ones.rename(columns={'Otif Ind': 'Y'})

# Calculate total count in column 'OTIL LINES' for each dealer code
df_total_count = df_OTIF_Stage.groupby('Dlr Cd Emer')['Otif Ind'].count().reset_index()
df_total_count = df_total_count.rename(columns={'Otif Ind': 'Grand Total'})

# # Merge the two dataframes on 'Dlr Cd Emer'
df_result = pd.merge(df_OTIF_Stage, df_count_ones, on='Dlr Cd Emer', how='left').fillna(0)
df_result = pd.merge(df_result, df_total_count, on='Dlr Cd Emer', how='left').fillna(0)


# Fill NaN values with 0 in case there are no '1' occurrences for a dealer code
df_result = df_result.fillna(0)

# Calculate the Percentage column
df_result['Percentage'] = (df_result['Y'] / df_result['Grand Total']) * 100

# Convert 'Dlr Cd Emer' to string explicitly
df_result['Dlr Cd Emer'] = df_result['Dlr Cd Emer'].astype(str)

In [28]:
# # Calculate count of 'Y' in column 'OTIL LINES' for each dealer code
# df_count_ones = df_OTIF_Stage[df_OTIF_Stage['Otif Ind'] == 'Y'].groupby('Dlr Cd Emer')['Otif Ind'].count().reset_index()
# df_count_ones = df_count_ones.rename(columns={'Otif Ind': 'Y'})

# # Calculate total count in column 'OTIL LINES' for each dealer code
# df_total_count = df_OTIF_Stage.groupby('Dlr Cd Emer')['Otif Ind'].count().reset_index()
# df_total_count = df_total_count.rename(columns={'Otif Ind': 'Grand Total'})

# # Merge the two dataframes on 'Dlr Cd Emer'
# df_result = pd.merge(df_count_ones, df_total_count, on='Dlr Cd Emer', how='right')

# # Fill NaN values with 0 in case there are no '1' occurrences for a dealer code
# df_result = df_result.fillna(0)

# # Calculate the Percentage column
# df_result['Percentage'] = (df_result['Y'] / df_result['Grand Total']) * 100

# # Convert 'Dlr Cd Emer' to string explicitly
# df_result['Dlr Cd Emer'] = df_result['Dlr Cd Emer'].astype(str)

In [29]:
df_result

,Dlr Cd Emer,Y,Grand Total,Percentage
0,E250,9758,10443,93.440582
1,E251,695,731,95.075239
2,E252,2729,2797,97.568824
3,E253,1699,1914,88.766980
4,E254,680,740,91.891892
5,E255,41,44,93.181818
6,E257,309,338,91.420118
7,E258,343,371,92.452830
8,E25D,36,42,85.714286
9,E25E,35,42,83.333333


In [30]:
# Specify data types for each column
dtype_mapping = {
    'Dlr Cd Emer': VARCHAR,
    'Y': INTEGER,
    'Grand Total': INTEGER,
    'Percentage': FLOAT,
}


In [31]:
# Convert the DataFrame to a table in the dbo schema without explicit data types
df_result.to_sql('OTIF_Stage', con=engine, index=False, if_exists='replace', schema='dbo')

-1

In [32]:
# Define the SQL query to create the final table
sql_create_table_query = """
CREATE TABLE [Targit_DM].[dbo].[OTIF_data] (
    [Dealer_Code] VARCHAR(25),
    [Rollup_Store] VARCHAR(10),
    [Y] INT,
    [Grand_Total] INT,
    [Percentage] FLOAT
)
"""

In [33]:
# Define the SQL query to insert into the final table
sql_insert_query = """
INSERT INTO [Targit_DM].[dbo].[OTIF_data]
SELECT 
       t1.[Dlr Cd Emer]
      ,t2.[Rollup_Store]
      ,t1.[Y]
      ,t1.[Grand Total]
      ,t1.[Percentage]
FROM [dbo].[OTIF_Stage] t1
LEFT JOIN [Daisy\Wagner].[XREF].[dbo].[Parts_KPIDealerCodeStoreRollup] t2
ON t1.[Dlr Cd Emer] = t2.Dealer_Code
"""

In [34]:
# Define the SQL query to truncate the staging table
sql_truncate_staging = "TRUNCATE TABLE [Targit_DM].[dbo].[OTIF_Stage]"

In [35]:
# Execute the SQL queries
with engine.connect() as connection:
    # Create the final table
    connection.execute(text(sql_create_table_query))
    
    # Insert into final table
    connection.execute(text(sql_insert_query))
    
    # Truncate staging table
    connection.execute(text(sql_truncate_staging))

    # Commit the transaction
    connection.commit()


In [ ]:
# Write df_OTIL to SQL Server (replace 'final_table' with your actual table name)
# df_OTIL.to_sql('Parts_OTIL', con=engine, index=False, if_exists='replace')


# Write df_OTIL to Excel
df_OTIL.to_excel('OTIL_Testing.xlsx', index=False)